# Import modelflow

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from ipywidgets import interact,Dropdown,Checkbox
from IPython.display import display, clear_output,Latex, Markdown
import pandas as pd

import modeljupyter as mj
from modelvis import waterplot


from modelclass import model
model.widescreen()
model.scroll_off()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;
}

<IPython.core.display.Javascript object>

# Load model and data, and run 

In [3]:
if not 'masia' in locals():
    print('Load model')
    masia,baseline = model.modelload('Asia.pcim',run=1,silent=1)

Load model


In [ ]:
xx = masia(baseline,stats=1,ljit=0,chunk=10)

Compile prolog:  56%|███████████████████████████████████████▉                               | 150/267  1.51s/code chunk

In [ ]:
from inject import inject
inject(masia)

In [ ]:
co2_input_var = [var for var in masia.totgraph.predecessors('WLD_CO2') if var.endswith('CO2')]

In [ ]:
co2_baseline = baseline.loc[:,co2_input_var].pipe(lambda df:df.rename(columns={c : masia.iso_dict.get(cname := c.split('_')[0],cname) for c in df.columns}))

# Variables to inputwidget

In [ ]:
outvar = 'GDI GCARBR CO2 LNN'.split()
displayvar = 'WLD_CO2 ASP_CO2 G20_CO2 '

#  masia.countries_GCARBR_A is list of all countries in the model with a countries_GCARBR_A 
#  masia.countries_ASEAN is list of all countries in ASEAN 
countries_rest = [country for country in masia.countries_GCARBR_A if country not in masia.countries_ASEAN]

slidedef_allasean =  {'All ASEAN countries' : {'var': ' '.join([f'{country}_GCARBR_A' for country in masia.countries_ASEAN ])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_allrest =  {'Rest of the world' : {'var': 
    ' '.join([f'{country}_GCARBR_A' for country in countries_rest])
                          , 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10,'dec':0}}
slidedef_countries = {masia.iso_dict.get(country,country)+', Extra' : 
                      {'var': f'{country}_GCARBR_A', 'value': 0.0,'min':-40,'max':40,'op':'+impulse','step':10,'dec':0} 
                      for country in  masia.countries_ASEAN}

slidedef = {**slidedef_allasean,**slidedef_allrest,**slidedef_countries}

In [ ]:
masia['wld_co2'].rename().plot(colrow=1,title='Baseline $CO^2$ emmision',top=0.9,ysize=5);

# Emmision with no policy 
Countries where the emmision is below the threshold are all attributed to the "small" label

In [ ]:
@interact(Threshold = (0,3000,100))
def explain(Periode   = Dropdown(options = masia.current_per,value=2049),
            Threshold =700):

    waterplot(co2_baseline.loc[[Periode],:].T,autosum=1,threshold = Threshold,allsort=1,title = '$CO^2$ emmision in',top=0.88);

# Make some policy 

In [ ]:
inject(masia)
input = masia.inputwidget_asia(2021,2050,slidedef=slidedef
                   ,showout=True,varpat=displayvar,showvar=True,go_now=1)

# Where does the reduction originate
The threshold leved and year can be chosen.
Countries where the reduction is below the threshold are all attributed to the "small" label  

In [ ]:

@interact(Threshold = (0,300,10))
def explain(Periode   = Dropdown(options = masia.current_per,value=2049),
            Threshold =200):
    masia.dekomp_plot('WLD_CO2',pct=0,threshold=Threshold,nametrans=masia.country_get)
    masia.dekomp_plot_per('WLD_CO2',pct=0,threshold=Threshold,per=Periode,sort=True,nametrans=masia.country_get)


In [ ]:
print(masia.get_eq_des('USA_EC',show_all=1))

In [ ]:
masia.USA_EC.tracepre(HR=0)

# Inversion of the model

In [ ]:
instruments = [[f'{country}_GCARBR_A' for country in masia.countries_GCARBR_A]]
instruments

In [ ]:
target = pd.DataFrame([36_440],index=[2049],columns=['WLD_CO2'])
target

In [ ]:
delay = 2049-2021
delay 

In [ ]:
from modelinvert import targets_instruments

In [ ]:
t_i = targets_instruments(databank=baseline,targets=target,instruments=instruments,model=masia,DefaultImpuls=50.,defaultconv=20,varimpulse=True)
t_i.targets

In [ ]:
t_i.jacobi(2049,28)

In [ ]:
t_i.debug=0
res = t_i(delay=29,varimpulse=True,silent=0)

In [ ]:
masia.calculate_freq_list(masia.coreorder)

In [ ]:
masia.flop_get

In [ ]:
masia.wld_co2

In [ ]:
masia.exodif()